In [ ]:
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
# marker style
marker01=dict(size=5, symbol='x-thin', line=dict(width=1, color='#1f77b4'))
marker02=dict(size=5, symbol='x-thin', line=dict(width=1, color='#aec7e8'))
marker03=dict(size=5, symbol='x-thin', line=dict(width=1, color='#ff7f0e'))
marker04=dict(size=5, symbol='x-thin', line=dict(width=1, color='#ffbb78'))
marker05=dict(size=5, symbol='x-thin', line=dict(width=1, color='#2ca02c'))
marker06=dict(size=5, symbol='x-thin', line=dict(width=1, color='#98df8a'))
marker07=dict(size=5, symbol='x-thin', line=dict(width=1, color='#d62728'))
marker08=dict(size=5, symbol='x-thin', line=dict(width=1, color='#ff9896'))
marker09=dict(size=5, symbol='x-thin', line=dict(width=1, color='#9467bd'))
marker10=dict(size=5, symbol='x-thin', line=dict(width=1, color='#c5b0d5'))
marker11=dict(size=5, symbol='x-thin', line=dict(width=1, color='#8c564b'))
marker12=dict(size=5, symbol='x-thin', line=dict(width=1, color='#c49c94'))

# N-body problem

In [ ]:
class three_body_model:

    def __init__(self):
        self.m0 = 1.00000597682
        self.m1 = 9.54786104043e-4
        self.m2 = 2.85583733151e-4
        self.G = 2.95912208286e-4

    def fcn(self, t, y):
        q = y[0:9]
        p = y[9:18]
        q_dot = self.H_p(p)
        p_dot = -self.H_q(q)
        return np.concatenate((q_dot, p_dot))

    def H_p(self, p):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        Hp = np.zeros(9)
        Hp[0:3] = p[0:3]/m0
        Hp[3:6] = p[3:6]/m1
        Hp[6:9] = p[6:9]/m2
        return Hp
    
    def H_q(self, q):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        Hq = np.zeros(9)
        Hq[0:3] = ( G*m0*m1*((q[0:3]-q[3:6])/np.power(np.linalg.norm(q[0:3]-q[3:6]),3)) 
                   +G*m0*m2*((q[0:3]-q[6:9])/np.power(np.linalg.norm(q[0:3]-q[6:9]),3)))
        Hq[3:6] = ( G*m1*m0*((q[3:6]-q[0:3])/np.power(np.linalg.norm(q[3:6]-q[0:3]),3))
                   +G*m1*m2*((q[3:6]-q[6:9])/np.power(np.linalg.norm(q[3:6]-q[6:9]),3)))
        Hq[6:9] = ( G*m2*m0*((q[6:9]-q[0:3])/np.power(np.linalg.norm(q[6:9]-q[0:3]),3)) 
                   +G*m2*m1*((q[6:9]-q[3:6])/np.power(np.linalg.norm(q[6:9]-q[3:6]),3)))
        return Hq

    def hamiltonian(self, y):
    
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        nt = y.shape[1]
        neq = y.shape[0]
        q = y[0:neq//2]
        p = y[neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            ham[i] = ( 0.5*( (1/m0)*np.dot(p[0:3,i],p[0:3,i]) 
                            +(1/m1)*np.dot(p[3:6,i],p[3:6,i]) 
                            +(1/m2)*np.dot(p[6:9,i],p[6:9,i]) ) 
                      - G *( (m0*m1)/np.linalg.norm(q[0:3,i]-q[3:6,i]) 
                            +(m0*m2)/np.linalg.norm(q[0:3,i]-q[6:9,i])
                            +(m1*m2)/np.linalg.norm(q[3:6,i]-q[6:9,i]) ) )

        return ham

In [ ]:
#################################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#################################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)

    return ode_result(y, t)

#################################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*fcn(tip1, uip1)

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*fcn(tn, yn)
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        y[:,it+1] = fsolve(g, y0, (yn, tn+dt))

    return ode_result(y, t)

#################################################################
def symplectic_euler(tini, tend, nt, yini, H_p, H_q):
    # This version only works for separable Hamiltionians (H(p,q)=T(p)+U(q)),
    # which allows us to remove the implicit terms

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        q[:,it+1] = q_n + dt*H_p(p_n)
        p[:,it+1] = p_n - dt*H_q(q[:,it+1])

    return ode_result(y,t)

#################################################################
def stormer_verlet(tini, tend, nt, yini, H_p, H_q):
    
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array
    
    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        p_np05 = p_n - (dt/2)*H_q(q_n)
        q[:,it+1] = q_n + dt*H_p(p_np05)
        p[:,it+1] = p_np05 - (dt/2)*H_q(q[:,it+1])
        
    return ode_result(y, t)

#################################################################
def optimized_815(tini, tend, nt, yini, H_p, H_q):
    
    def stormer_verlet_step(dt, yini, H_p, H_q):
        q_n, p_n = np.split(yini,2)
        p_np05 = p_n - (dt/2)*H_q(q_n)
        q_np1  = q_n + dt*H_p(p_np05)
        p_np1  = p_np05 - (dt/2)*H_q(q_np1)
        return np.concatenate((q_np1, p_np1))
     
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    nstep = 15
    gamma = np.zeros(nstep+1)
    gamma[0]  =  0.
    gamma[1]  =  0.74167036435061295344822780
    gamma[2]  = -0.40910082580003159399730010
    gamma[3]  =  0.19075471029623837995387626
    gamma[4]  = -0.57386247111608226665638773
    gamma[5]  =  0.29906418130365592384446354
    gamma[6]  =  0.33462491824529818378495798
    gamma[7]  =  0.31529309239676659663205666
    gamma[8]  = -0.79688793935291635401978884
    gamma[9]  = gamma[7]
    gamma[10] = gamma[6]
    gamma[11] = gamma[5]
    gamma[12] = gamma[4]
    gamma[13] = gamma[3]
    gamma[14] = gamma[2]
    gamma[15] = gamma[1]
                          
    ytmp = y[:,0]

    for it in range(nt-1):
 
        for istep in range(nstep):

            ti = 0.0
            te = gamma[istep+1]*dt
            ytmp = stormer_verlet_step(te-ti, ytmp, H_p, H_q)

        y[:,it+1] = ytmp
        
    return ode_result(y, t)

## Integration for a short time 

In [ ]:
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.
qini[1] =  0.
qini[2] =  0.
qini[3] = -3.5023653
qini[4] = -3.8169847
qini[5] = -1.5507963
qini[6] =  9.0755314
qini[7] = -3.0458353
qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.
pini[1] =  0.
pini[2] =  0.
pini[3] =  0.00565429*m1
pini[4] = -0.00412490*m1
pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2
pini[7] =  0.00483525*m2
pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 12500.
nt = 2001

tbm = three_body_model()
H_p = tbm.H_p
H_q = tbm.H_q
fcn = tbm.fcn
hamiltonian = tbm.hamiltonian

# forward euler
sol_fe = forward_euler(tini, tend, nt, yini, fcn)
yfe = sol_fe.y
ham_fe = hamiltonian(sol_fe.y)

# backward euler
sol_be = backward_euler(tini, tend, nt, yini, fcn)
ybe = sol_be.y
ham_be = hamiltonian(sol_be.y)

# symplectic Euler 
sol_se = symplectic_euler(tini, tend, nt, yini, H_p, H_q)
yse = sol_se.y
ham_se = hamiltonian(sol_se.y)

# Stormer-Verlet 
sol_sv = stormer_verlet(tini, tend, nt, yini, H_p, H_q)
ysv = sol_sv.y
ham_sv = hamiltonian(sol_sv.y)

# optimized 8-15
sol_o815 = optimized_815(tini, tend, nt, yini, H_p, H_q)
yo815 = sol_o815.y
ham_o815 = hamiltonian(sol_o815.y)

# Dopri 853 integration
tol = 1e-6
sol_dop853 = solve_ivp(fcn, (tini, tend), yini, method='DOP853', atol=tol, rtol=tol)
ydop853 = sol_dop853.y
ham_dop853 = hamiltonian(sol_dop853.y)

# plot phase plan
fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=yfe[3]-yfe[0], y=yfe[4]-yfe[1], mode='markers', marker=marker01, 
                             name='Forward euler', legendgroup='fe'))
fig_sol.add_trace(go.Scatter(x=yfe[6]-yfe[0], y=yfe[7]-yfe[1], mode='markers', marker=marker02, 
                             showlegend=False, legendgroup='fe'))
fig_sol.add_trace(go.Scatter(x=ybe[3]-ybe[0], y=ybe[4]-ybe[1], mode='markers', marker=marker03, 
                             name='Backward euler', legendgroup='be'))
fig_sol.add_trace(go.Scatter(x=ybe[6]-ybe[0], y=ybe[7]-ybe[1], mode='markers', marker=marker04, 
                             showlegend=False, legendgroup='be'))
fig_sol.add_trace(go.Scatter(x=yse[3]-yse[0], y=yse[4]-yse[1], mode='markers', marker=marker05, 
                             name='Symplectic euler', legendgroup='se'))
fig_sol.add_trace(go.Scatter(x=yse[6]-yse[0], y=yse[7]-yse[1], mode='markers', marker=marker06, 
                             showlegend=False, legendgroup='se'))
fig_sol.add_trace(go.Scatter(x=ysv[3]-ysv[0], y=ysv[4]-ysv[1], mode='markers', marker=marker07, 
                             name='Stormer-Verlet', legendgroup='sv'))
fig_sol.add_trace(go.Scatter(x=ysv[6]-ysv[0], y=ysv[7]-ysv[1], mode='markers', marker=marker08, 
                             showlegend=False, legendgroup='sv'))
fig_sol.add_trace(go.Scatter(x=yo815[3]-yo815[0], y=yo815[4]-yo815[1], mode='markers', marker=marker09, 
                             name='Optimized 8-15', legendgroup='o815'))
fig_sol.add_trace(go.Scatter(x=yo815[6]-yo815[0], y=yo815[7]-yo815[1], mode='markers', marker=marker10, 
                             showlegend=False, legendgroup='o815'))
fig_sol.add_trace(go.Scatter(x=ydop853[3]-ydop853[0], y=ydop853[4]-ydop853[1], mode='markers', marker=marker11, 
                             name='Dopri 853', legendgroup='dop853'))
fig_sol.add_trace(go.Scatter(x=ydop853[6]-ydop853[0], y=ydop853[7]-ydop853[1], mode='markers', marker=marker12, 
                             showlegend=False, legendgroup='dop853'))
fig_sol.update_layout(title='Solution in xy plan', xaxis_title='x', yaxis_title='y', margin=dict(b=0))
fig_sol.show()

# plot Hamiltonian
fig_ham = go.Figure()
fig_ham.add_trace(go.Scatter(x=sol_fe.t, y=ham_fe, mode='markers', marker=marker01, name='Forward euler'))
fig_ham.add_trace(go.Scatter(x=sol_be.t, y=ham_be, mode='markers', marker=marker03, name='Backward euler'))
fig_ham.add_trace(go.Scatter(x=sol_se.t, y=ham_se, mode='markers', marker=marker05, name='Symplectic euler'))
fig_ham.add_trace(go.Scatter(x=sol_sv.t, y=ham_sv, mode='markers', marker=marker07, name='Stormer-Verlet'))
fig_ham.add_trace(go.Scatter(x=sol_o815.t, y=ham_o815, mode='markers', marker=marker09, name='Optimized 8-15'))
fig_ham.add_trace(go.Scatter(x=sol_dop853.t, y=ham_dop853, mode='markers', marker=marker11, name='Dopri 853'))
fig_ham.update_layout(title='Hamiltonian', xaxis_title='t', yaxis=dict(title='Hamiltonian', exponentformat = 'e'))
fig_ham.show()

## Integration for a longer time

**La cellule suivante met longtemps à s'exécuter, c'est normal**

In [ ]:
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.
qini[1] =  0.
qini[2] =  0.
qini[3] = -3.5023653
qini[4] = -3.8169847
qini[5] = -1.5507963
qini[6] =  9.0755314
qini[7] = -3.0458353
qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.
pini[1] =  0.
pini[2] =  0.
pini[3] =  0.00565429*m1
pini[4] = -0.00412490*m1
pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2
pini[7] =  0.00483525*m2
pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 9500000.
nt = 30001

tbm = three_body_model()
H_p = tbm.H_p
H_q = tbm.H_q
fcn = tbm.fcn
hamiltonian = tbm.hamiltonian

# Stormer-Verlet 
sol_sv = stormer_verlet(tini, tend, nt, yini, H_p, H_q)
ysv = sol_sv.y
ham_sv = hamiltonian(sol_sv.y)

# RK45 integration
tol = 1e-6
sol_rk45 = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=tol, rtol=tol)
yrk45 = sol_rk45.y
ham_rk45 = hamiltonian(sol_rk45.y)

# optimized 8-15
sol_o815 = optimized_815(tini, tend, nt, yini, H_p, H_q)
yo815 = sol_o815.y
ham_o815 = hamiltonian(sol_o815.y)

# Dopri 853 integration
tol = 1e-6
sol_dop853 = solve_ivp(fcn, (tini, tend), yini, method='DOP853', atol=tol, rtol=tol)
ydop853 = sol_dop853.y
ham_dop853 = hamiltonian(sol_dop853.y)

nfactor = 100
marker01=dict(size=5, symbol='x-thin', line=dict(width=1, color='#1f77b4'), maxdisplayed=nt//nfactor)
marker02=dict(size=5, symbol='x-thin', line=dict(width=1, color='#aec7e8'), maxdisplayed=nt//nfactor)
marker07=dict(size=5, symbol='x-thin', line=dict(width=1, color='#d62728'), maxdisplayed=nt//nfactor)
marker08=dict(size=5, symbol='x-thin', line=dict(width=1, color='#ff9896'), maxdisplayed=nt//nfactor)
marker09=dict(size=5, symbol='x-thin', line=dict(width=1, color='#9467bd'), maxdisplayed=nt//nfactor)
marker10=dict(size=5, symbol='x-thin', line=dict(width=1, color='#c5b0d5'), maxdisplayed=nt//nfactor)
marker11=dict(size=5, symbol='x-thin', line=dict(width=1, color='#8c564b'), maxdisplayed=nt//nfactor)
marker12=dict(size=5, symbol='x-thin', line=dict(width=1, color='#c49c94'), maxdisplayed=nt//nfactor)

# plot phase plan
fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=ysv[3]-ysv[0], y=ysv[4]-ysv[1], mode='markers', marker=marker07, 
                             name='Stormer-Verlet', legendgroup='sv'))
fig_sol.add_trace(go.Scatter(x=ysv[6]-ysv[0], y=ysv[7]-ysv[1], mode='markers', marker=marker08, 
                             showlegend=False, legendgroup='sv'))
fig_sol.add_trace(go.Scatter(x=yrk45[3]-yrk45[0], y=yrk45[4]-yrk45[1], mode='markers', marker=marker01, 
                             name='RK45', legendgroup='rk45'))
fig_sol.add_trace(go.Scatter(x=yrk45[6]-yrk45[0], y=yrk45[7]-yrk45[1], mode='markers', marker=marker02, 
                             showlegend=False, legendgroup='rk45'))
fig_sol.add_trace(go.Scatter(x=yo815[3]-yo815[0], y=yo815[4]-yo815[1], mode='markers', marker=marker09, 
                             name='Optimized 8-15', legendgroup='o815'))
fig_sol.add_trace(go.Scatter(x=yo815[6]-yo815[0], y=yo815[7]-yo815[1], mode='markers', marker=marker10, 
                             showlegend=False, legendgroup='o815'))
fig_sol.add_trace(go.Scatter(x=ydop853[3]-ydop853[0], y=ydop853[4]-ydop853[1], mode='markers', marker=marker11, 
                             name='Dopri 853', legendgroup='dop853'))
fig_sol.add_trace(go.Scatter(x=ydop853[6]-ydop853[0], y=ydop853[7]-ydop853[1], mode='markers', marker=marker12, 
                             showlegend=False, legendgroup='dop853'))
fig_sol.update_layout(title='Solution in xy plan', xaxis_title='x', yaxis_title='y', margin=dict(b=0))
fig_sol.show()

# plot Hamiltonian
fig_ham = go.Figure()
fig_ham.add_trace(go.Scatter(x=sol_sv.t, y=ham_sv, mode='markers', marker=marker07, name='Stormer-Verlet'))
fig_ham.add_trace(go.Scatter(x=sol_rk45.t, y=ham_rk45, mode='markers', marker=marker01, name='RK45'))
fig_ham.add_trace(go.Scatter(x=sol_o815.t, y=ham_o815, mode='markers', marker=marker09, name='Optimized 8-15'))
fig_ham.add_trace(go.Scatter(x=sol_dop853.t, y=ham_dop853, mode='markers', marker=marker11, name='Dopri 853'))
fig_ham.update_layout(title='Hamiltonian', xaxis_title='t', yaxis=dict(title='Hamiltonian', exponentformat = 'e'))
fig_ham.show()